In [8]:
pip install keras==2.4.3 "h5py<3.0"

  Using cached Keras-2.4.3-py2.py3-none-any.whl (36 kB)
  Attempting uninstall: keras
    Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /private/var/folders/tt/kzfjlh_x0535tls5t23lpymr0000gn/T/pip-req-build-mv7anrtl
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101064 sha256=a6825550ac8a4cbabd0afcfbc139f0d27da07294c22c07454da431b140ebaf27
  Stored in directory: /private/var/folders/tt/kzfjlh_x0535tls5t23lpymr0000gn/T/pip-ephem-wheel-cache-uv_ot9pz/wheels/67/d2/f4/96ae3c3c62d1e05abfc8860ad0c1207794726d44ebbbb547f3
Successfully built keras-contrib
Note: you may need to restart the kernel to use updated packages.


In [1]:

import pandas as pd
import numpy as np

from tensorflow import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

from sklearn.metrics import accuracy_score,classification_report

In [2]:
# set parameters:
max_features = 6000
max_length = 400


(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train observations')
print(len(x_test), 'test observations')


25000 train observations
25000 test observations


In [3]:
#단어 대 숫자 매핑 생성
wind = imdb.get_word_index()
revind = dict((v,k) for k,v in wind.items())

#컴퓨터는 문자, 단어가 아닌 숫자로만 이해하고 작업가능!
print (x_train[0])
print (y_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 2, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
1


In [4]:
#숫자 매핑을 텍스트 형식으로 변환 후 단계
#딕셔너리를 사용해 정수 형식에서 텍스트 형식으로 맵을 역순 변환 
def decode(sent_list):
    new_words = []
    for i in sent_list:
        new_words.append(revind[i])
    comb_words = " ".join(new_words)
    return comb_words  

print (decode(x_train[0]))

the as you with out themselves powerful lets loves their becomes reaching had journalist of lot from anyone to have after out atmosphere never more room and it so heart shows to years of every never going and help moments or of every chest visual movie except her was several of enough more with is now current film as you of mine potentially unfortunately of you than him that with out themselves her get for was camp of you movie sometimes movie that with scary but and to story wonderful that in seeing in character to of 70s musicians with heart had shadows they of here that with her serious to have does when from why what have critics they is you that isn't one will very to as itself with other and in of seen over landed for anyone of and br show's to whether from than out themselves history he name half some br of and odd was two most of mean for 1 any an boat she he should is thought and but of script you not while history he heart to real at barrel but when from one bit then have two

In [ ]:
#효율적인 연산을 위한 패드 배열 

#최대길이가 400단어인 추가 문장을 더하기 위해 패딩작업 수행
#데이터는 신경망을 사용한 계산을 수행하기 점점 쉬움

x_train = sequence.pad_sequences(x_train, maxlen=max_length)
x_test = sequence.pad_sequences(x_test, maxlen=max_length)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

In [ ]:
# Deep Learning architecture parameters
batch_size = 32
embedding_dims = 60
num_kernels = 260
kernel_size = 3
hidden_dims = 300
epochs = 3

In [16]:
#모델 구축

model = Sequential()

model.add(Embedding(max_features,embedding_dims,input_length=max_length))
model.add(Dropout(0.2))

model.add(Conv1D(num_kernels,kernel_size,padding='valid',activation='relu',strides=1))
model.add(GlobalMaxPooling1D())

model.add(Dense(hidden_dims))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 400, 60)           360000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 400, 60)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 398, 260)          47060     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 260)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 300)               78300     
_________________________________________________________________
dropout_3 (Dropout)          (None, 300)               0         
_________________________________________________________________
activation_2 (Activation)    (None, 300)              

In [17]:
model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,validation_split=0.2)
#학습의 정확도 증가 하지만 유효성 검사의 정확도는 떨어진다
#에포크 수를 늘리는 것보다 모델의 정확도를 향상 시키는 다른 방법을 시도해야 한다.(모델 아키텍처 크기 늘리기)

Epoch 1/3
625/625 [==============================] - 16s 24ms/step - loss: 0.5715 - accuracy: 0.6487 - val_loss: 0.3019 - val_accuracy: 0.8754
Epoch 2/3
625/625 [==============================] - 16s 25ms/step - loss: 0.2475 - accuracy: 0.9017 - val_loss: 0.2938 - val_accuracy: 0.8842
Epoch 3/3
625/625 [==============================] - 16s 26ms/step - loss: 0.1512 - accuracy: 0.9442 - val_loss: 0.2971 - val_accuracy: 0.8898


In [18]:

import numpy as np

y_train_predclass = model.predict_classes(x_train,batch_size=batch_size)
y_test_predclass = model.predict_classes(x_test,batch_size=batch_size)

y_train_predclass.shape = y_train.shape
y_test_predclass.shape = y_test.shape


# Model accuracies & metrics calculation
print (("\n\nCNN 1D  - Train accuracy:"),(round(accuracy_score(y_train,y_train_predclass),3)))
print ("\nCNN 1D of Training data\n",classification_report(y_train, y_train_predclass))
print ("\nCNN 1D - Train Confusion Matrix\n\n",pd.crosstab(y_train, y_train_predclass,rownames = ["Actuall"],colnames = ["Predicted"]))      

print (("\nCNN 1D  - Test accuracy:"),(round(accuracy_score(y_test,y_test_predclass),3)))
print ("\nCNN 1D of Test data\n",classification_report(y_test, y_test_predclass))
print ("\nCNN 1D - Test Confusion Matrix\n\n",pd.crosstab(y_test, y_test_predclass,rownames = ["Actuall"],colnames = ["Predicted"]))      


/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '




CNN 1D  - Train accuracy: 0.964

CNN 1D of Training data
               precision    recall  f1-score   support

           0       0.95      0.98      0.96     12500
           1       0.98      0.95      0.96     12500

    accuracy                           0.96     25000
   macro avg       0.96      0.96      0.96     25000
weighted avg       0.96      0.96      0.96     25000


CNN 1D - Train Confusion Matrix

 Predicted      0      1
Actuall                
0          12271    229
1            683  11817

CNN 1D  - Test accuracy: 0.878

CNN 1D of Test data
               precision    recall  f1-score   support

           0       0.85      0.92      0.88     12500
           1       0.92      0.83      0.87     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000


CNN 1D - Test Confusion Matrix

 Predicted      0      1
Actuall                
0          11540   

In [ ]:
#학습의 정확도는 96프로인데 테스트 정확도느 87프로이다. 모델 과적합(오버피팅)때문